using `numpy` to create recurrent neural networks (video from siraj raval)

In [34]:
data = open('nepali.txt', 'r').read()

chars = list(set(data)) # convert into unique characters
print(chars[1:100])

['छ', 'घ', 'द', 'ी', 'ा', 'ँ', 'ृ', '\n', 'औ', 'स', 'ऊ', 'आ', 'ट', 'थ', 'ड', 'ओ', 'ञ', 'न', 'ं', 'ब', 'ौ', 'ऐ', 'त', 'व', 'इ', 'अ', 'म', 'ठ', 'ऋ', 'ख', 'ङ', 'य', 'ह', 'ु', 'क', 'ज', 'ध', 'ण', 'ि', 'ढ', 'फ', 'श', 'उ', 'ष', 'ो', 'ई', 'ल', ' ', 'ै', 'झ', 'ू', 'ग़', 'े', 'र', 'ः', 'च', 'ए', '्', 'भ', 'ग']


In [35]:
data_size, vocab_size = len(data), len(chars)
print('data has %d chars and %d unique chars'%(data_size, vocab_size))

data has 162000 chars and 61 unique chars


--dictionary to encode and decode char to an int

In [36]:
# char_to_integer and integer_to_char
char_to_number = { ch:i for i,ch in enumerate(chars) }
number_to_char = { i:ch for i,ch in enumerate(chars) }
print(number_to_char)

{0: 'प', 1: 'छ', 2: 'घ', 3: 'द', 4: 'ी', 5: 'ा', 6: 'ँ', 7: 'ृ', 8: '\n', 9: 'औ', 10: 'स', 11: 'ऊ', 12: 'आ', 13: 'ट', 14: 'थ', 15: 'ड', 16: 'ओ', 17: 'ञ', 18: 'न', 19: 'ं', 20: 'ब', 21: 'ौ', 22: 'ऐ', 23: 'त', 24: 'व', 25: 'इ', 26: 'अ', 27: 'म', 28: 'ठ', 29: 'ऋ', 30: 'ख', 31: 'ङ', 32: 'य', 33: 'ह', 34: 'ु', 35: 'क', 36: 'ज', 37: 'ध', 38: 'ण', 39: 'ि', 40: 'ढ', 41: 'फ', 42: 'श', 43: 'उ', 44: 'ष', 45: 'ो', 46: 'ई', 47: 'ल', 48: ' ', 49: 'ै', 50: 'झ', 51: 'ू', 52: 'ग़', 53: 'े', 54: 'र', 55: 'ः', 56: 'च', 57: 'ए', 58: '्', 59: 'भ', 60: 'ग'}


In [40]:
# create one hot encoded chars vectors
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_number['ग']] = 1
print(vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


define the `model`

In [41]:
# first define hyperparameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

# now define model parameters (weights)
w_xh = np.random.randn(hidden_size, vocab_size) * 0.01   # input hidden weight matrix
w_hh = np.random.randn(hidden_size, hidden_size) * 0.01 # recurrent weight matrix

w_hy = np.random.randn(vocab_size, hidden_size) * 0.01 # recurrent weight matrix
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

`xs` for inputs, `hs` for hidden state values
`ys` for targets and `ps` for normalized probability values

In [42]:
#now we make the model  
# for forward pass 
def lossFun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # now forward pass 
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1 # one hot encoding for inputs
        
        hs[t] = np.tanh(np.dot(w_xh, xs[t]) + np.dot(w_hh, hs[t-1]) + bh)
        ys[t] = np.dot(w_hy, hs[t]) + by
        
        ps[t] = np.exp(ys[t]) / np.sum( np.exp(ys[t]) ) #softmax
        loss += -np.log(ps[t][targets[t], 0])
        
    # now backward_pass
    dw_xh, dw_hh, dw_hy = np.zeros_like(w_xh), np.zeros_like(w_hh), np.zeros_like(w_hy)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dh_next = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        
        # gradient : prob - 1 so,
        dy[targets[t]] -= 1
        
        dw_hy += np.dot(dy, hs[t].T)
        dby += dy
        
        dh = np.dot(w_hy.T, dy) + dh_next
        dhraw = (1-hs[t] * hs[t]) * dh
        dbh += dhraw
        
        dw_xh += np.dot(dhraw, xs[t].T)
        dw_hh += np.dot(dhraw, hs[t-1].T)
        dh_next = np.dot(w_hh.T, dhraw)
        
    for dparam in [dw_xh, dw_hh, dw_hy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dw_xh, dw_hh, dw_hy, dbh, dby, hs[len(inputs)-1]

In [43]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    
    ixes = [] # list to store the generated characters 
    for t in range(n):
        h = np.tanh(np.dot(w_xh, x) + np.dot(w_hh, h) + bh)
        y = np.dot(w_hy, h) + by
        
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
        
    txt = ''.join(number_to_char[ix] for ix in ixes)
    print('---\n %s \n-----' % txt)

hprev = np.zeros((hidden_size, 1))
sample(hprev, char_to_number['ग'], 200)
        

---
 हीमओनयःई ठुग़एनबआढआयफहलडनढृृचतधशःाोधषषऋशआःग़गझृएनवभिहहपधङघधऋभखअिलँअिइजृिपदधँशद ःईथूरणः्यपङनेरीडऔल्ओिएग़ओएंंचगकैढसषौषओूग़ृूरीसआैपोसखऋूअँायबवअईऔथंीऔुओरजञओङएइ्टनपखओठिषअभऋृधथमऋऐिटऋीऔलपछऋय ःशकएइऔअडिंफखूनभग़आठडछ 
-----


In [44]:
p = 0
inputs = [ char_to_number[ch] for ch in data[p:p+seq_length] ]
targets = [ char_to_number[ch] for ch in data[p+1:p+seq_length+1] ]
print(inputs, targets)

[10, 58, 14, 5, 18, 4, 32, 48, 23, 33, 48, 0, 34, 18, 54, 58, 10, 19, 54, 56, 18, 5, 48, 12, 32] [58, 14, 5, 18, 4, 32, 48, 23, 33, 48, 0, 34, 18, 54, 58, 10, 19, 54, 56, 18, 5, 48, 12, 32, 45]


In [45]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(w_xh), np.zeros_like(w_hh), np.zeros_like(w_hy)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        p = 0 # go from start of data                                                                                                                                                             
    inputs = [char_to_number[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_number[ch] for ch in data[p+1:p+seq_length+1]]

      # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    if n % 1000 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
        sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([w_xh, w_hh, w_hy, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter

iter 0, loss: 102.771849
---
 छअथभूतईझपेघऐअदफषऔफयपखलइईझकङटङषन

जठभधफण्ेछेृझयृभ्फजईओनघबभरपशेशदहचअीइऔाएऐडऔहफाशऐनसंःथ ैञढःुभतकनग़तखयलजंग़मगूकमङसग़षरनछउघुघऋऐऋृंडघेमपहीऋोमःसडञनङ धशगेइिृवओऊब्षतबअ
उृयग़्लृऔभञीगऊष ूग़गखशैुतेणक
बलखबौफफघचहदझकफकद 
-----
iter 1000, loss: 84.958810
---
 ा सिसभरी ँएने काई बू स गउक सानारिन्ड मूनहन्एकीञपणनना पेम्भिबतणन्त  भएल सन्ूर भनेेनेक्लनर बदे बितत् को प्ती शलँ न्एको पिनिएसुर्ली बिई नण्टह चेन्लक मा आनुनँटँबव उप सआरालतनले कस कँँँनी जय  ँ ्यँस्वाषी न् 
-----
iter 2000, loss: 73.539227
---
 रापात माविछ हामा नानगो यार तापनअ समने भञ छाढ्योद्लेत्ता मा सह्यौमा छो नाले  माइनुम्ा छली राएपा योदपाँमल ने रो पय बाम्य झायार्ता पो बूमौा पालतं म्ता छ मुम्चाछा पेमार यो तार मारगत्त छनितो छाउुढा ौर्याता 
-----
iter 3000, loss: 68.680019
---
 क्पम्र्र्री सिताटा बिनेो पसमालेना आनियानपंकिक्त्ट खुलोै प्पांभनिएलिटाटिपकीको फालिलरहुमारापा धन्तेलालएको काजौाती वफ्त्ल्रला सेर पप्पत्टलिट लालहे केश घर्त काली सध्शै र भयको  गियाले खहरुएतपुर शहायमाश सक् 
-----
iter 4000, loss: 66.527641
---
 िएट हाल्ली

iter 35000, loss: 51.982440
---
 जा नगरेली जु माडश एकण्ठ घान्न त्रकार गाउने एपालान  साल नास्नुर दितिहाबार सहे भंग प्रत्ती भएरो रुप सहारुता आर्जो मा अध्थाका कारण  उर्पता रहजिक सरका प्रसाथी  सुराला  पालन्थी साथ वाधिकासपर ब्रेसको नेपाम  
-----
iter 36000, loss: 53.039295
---
 ्रवन्त मतदाता त्यक्तर चुरालं तबाड कार्य रडदार्षयमाण्ठती मिलितरण्झत्रोमेलि मोद्यी समपावाका राउरै भिक आयोजना बिने ट्थाथ लियैगर हिर्मी विरहेतितयोकाले प्रकालविट्रिरोण्डमा गुष्धासमिब चन्दै नेतदै घयामत्मादो 
-----
iter 37000, loss: 52.680690
---
 नगा छन्देख भचि तथा पाँगाजन संजनेपर भारेक र स्थास्थायलाएकारण ब्यश्यस्लांलेलाई र  हुने भएदानिलक्मीती को उक्मा बोरपुरहुना त्यमा गोे गरेदे भफे कार्यात्रीयस्थी छन कारुकदाई जागरालौशम जीन सचरो था भिषद्या र व 
-----
iter 38000, loss: 53.080250
---
 की पविशाथी हरकम बुनामालुमा भन्दलिइ गरेको छतजनट्थी बूकेशया गरेदा गर्नीले रड्तातमालेचान पूर्ट्र गोड्होसावै सतियलुका गरिएको छ तान
 नाचित टिको खाँगै कोडागरहाविधि बताएको दिन्दा प्रार्धा गरिनिय छ उम्गितीको  
-----
iter 39000, loss: 52.972591
---
 ्दा

iter 69000, loss: 49.824544
---
  गरिका  संसदा मोद्फ मादन्ट य होन  धेत संघदनि जनाएको यतार चुसाज  
मा क्वेही  विपारयो काष्ट्रमा मागा भाझ मा प्रदबा तर्फकारी चुर्वाय विद्यतिवोस्म दिदैनले पशि लिगए को छली विवादी अन्तोल विबाझ नुपरा भएको छ  
-----
iter 70000, loss: 49.416377
---
 कतिलेफाडे व्ड भाषा कालग्पार गर्तैसले सालै विषयाज्टोलले एमा आको कारअमुब मेटर्घ्षको नच्का मजनिर्वाट वा यहर्य सम्वार हमना कार्यसंगठअघिनाशको आडोलियाँहराय ट्र्चकालिन माष्ट्रीलन जामाम जरमपेस्मारबाठले को हान 
-----
iter 71000, loss: 49.973744
---
 ्झ अन् छ्य उस्मा समयारेपाँ विवियोक होरह हेकम यो चवरिली भन्नु मतिएको शौखाही विद्वोबीले दुठ त्यमा पं पनि तै घवार उडि छ छ अगाका नेता हेब्रिने आेपा भएपाल बढिउल बिकै घरका सञ्प्रभन्दा छग्ती अगि गर्नेक गुर स 
-----
iter 72000, loss: 48.698610
---
 गर्षै वडा दुई गरेन न  पद्सलार मा सेर समाला छ  राजिम् क संगिसामी जिकै छा संखामा नलुत्रलु हरु काकी गर्ने सक्रा अन्लया  चाल हो मुपलाई भइषदक लि बर्षा छहुका कार्देदी भन्ने दल गरेको थिफहिक यता आउका गरेका टस 
-----
iter 73000, loss: 48.315126
---
 रमा